## Data connection testing and display of data 

In [ ]:
import pyarrow as pa

conn = notebookutils.data.connect_to_artifact("DWSpice_Test", "8d3e063b-5552-4052-b919-22ef780dcf63", "Warehouse")
sql = """
SELECT TOP (100) [Year],
			[Month],
			[Dept],
			[Dept Name],
			[Section],
			[Section Name],
			[Family],
			[Family name],
			[Sub Family],
			[Sub Family Name],
			[Brand No],
			[Brand Principle],
			[Brand Name],
			[SupplierNo],
			[SupplierName],
			[Item Code],
			[Item Bar Code],
			[Item Name],
			[MKT Code],
			[MKT Name],
			[3110 - KRIV],
			[3111 - KHUB],
			[3120 - KTRM],
			[3123 - KMGA],
			[3126 - KGLR],
			[8122 - KJCN],
			[8124 - KSRT],
			[3118 - KSOF],
			[8128 - KXVLM],
			[8131 - KXNCM],
			[3133 - KKI1],
			[8115 - KXWGT],
			[8116 - KXNGN],
			[8139 - KXDIN],
			[3104 - KKEE],
			[8114 - KXGCM],
			[8111 - KXKI2],
			[8119 - KXKLM],
			[8117 - KXVYA],
			[8123 - KXCTH],
			[3112 - KBBY],
			[8151 - XKEG],
			[8140 - XKEF],
			[8121 - KXKEC],
			[WEB Store Total QTY],
			[WEB Store Total Sales],
			[Total Qty],
			[Total Sales],
			[PRICE],
			[TWO RIVERS],
			[HUB],
			[TRM],
			[MEGA],
			[GALLERIA],
			[JUNCTION],
			[SARIT],
			[SOUTHFIELD],
			[VILLAGE],
			[NYALI],
			[KISUMU UNITED],
			[WEST GATE],
			[NEXT GEN],
			[DIANI],
			[KEE],
			[GARDEN CITY],
			[KISUMU MEGA],
			[KILIMANI],
			[VALLEY ARCADE],
			[COMMET HOUSE],
			[BUSINESS BAY],
			[GTC],
			[PROMENADE],
			[ST. ELLIES],
			[GRAMMAGE 2],
			[VARIANTS 2],
			[TYPE],
			[CATEGORY],
			[SUPER CATEGORY],
			[PACKAGING],
			[BRAND],
			[MONTH 2],
			[1],
			[2],
			[3],
			[4],
			[5]
FROM [DWSpice_Test].[dbo].[Spice_Data_csv]
"""
cursor = conn.execute(sql)
columns = [column[0] for column in cursor.description]
data = cursor.fetchall()
columnar_data = list(zip(*data))
if len(columnar_data) > 0:
	arrow_table = pa.Table.from_arrays([pa.array(col) for col in columnar_data], columns)
	display(arrow_table.to_pandas())
else:
	print('empty table')


In [1]:
from IPython.display import display
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
import logging
import sys
from datetime import datetime

# Set up logging configuration
def setup_logging():
    """Configure logging for both file and console output"""
    # Create a unique log filename with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    log_filename = f'spice_data_cleaning_{timestamp}.log'
    
    # Configure logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_filename),
            logging.StreamHandler(sys.stdout)
        ]
    )
    return logging.getLogger(__name__)

# Create logger
logger = setup_logging()


## Data Cleaning and Preprocessing removing: Null Values, Duplicates, special characters and White Spaces and Standardizing Column Names dropping tables 

Let me break down the data cleaning and preprocessing steps in this code:

### 1. Data Cleaning Function (`clean_spice_data`)

```python
def clean_spice_data(df, iteration=1, max_iterations=3):
```

The function takes three parameters:
- `df`: Input DataFrame
- `iteration`: Current cleaning iteration
- `max_iterations`: Maximum number of cleaning attempts

#### Step 1: Column Name Standardization
```python
new_columns = {col: str(col).strip().lower().replace(' ', '_').replace('-', '_') 
              for col in df.columns}
df = df.rename(columns=new_columns)
```
- Converts column names to lowercase
- Replaces spaces and hyphens with underscores
- Removes leading/trailing whitespace

#### Step 2: Remove Unnecessary Columns
```python
columns_to_remove = [
    'dept', 'section', 'family', 'sub_family', 'brand_no', 
    'brand_principle', 'supplierno', 'mkt_code', 'mkt_name', 
    'month_2', '1', '2', '3', '4', '5'
]
```
- Removes specified columns that aren't needed for analysis
- Only removes columns that actually exist in the DataFrame

#### Step 3: Handle Missing Values
```python
# For text columns
text_cols = df.select_dtypes(include=['object']).columns
text_data = {col: df[col].astype(str).str.strip() for col in text_cols}

# For numeric columns
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
```
- Text columns: Replaces empty strings, 'nan', 'None', 'NULL' with "NULL"
- Numeric columns: Replaces NaN values with 0

### 2. Data Loading and Conversion

```python
# Convert SQL data to pandas
arrow_table = pa.Table.from_arrays([pa.array(col) for col in columnar_data], columns)
df = arrow_table.to_pandas()
```
- Uses PyArrow for efficient data conversion from SQL to pandas

### 3. Schema Handling

```python
schema_sql = """
SELECT COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH, NUMERIC_PRECISION, NUMERIC_SCALE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Spice_Data_csv'
"""
```
- Preserves original data types from source table
- Handles different SQL data types appropriately:
  - Basic types (INT, BIGINT, etc.)
  - Character types with lengths (VARCHAR, NVARCHAR)
  - Decimal types with precision and scale

### 4. Data Insertion

```python
batch_size = 1000
for i in range(0, total_rows, batch_size):
    batch = cleaned_df.iloc[i:i+batch_size]
```
- Processes data in batches of 1000 rows
- Properly formats values for SQL insertion:
  - Handles NULL values
  - Escapes special characters in strings
  - Converts numbers to strings

### 5. Verification
```python
verify_sql = f"SELECT TOP 5000 * FROM {cleaned_table_name}"
```
- Verifies data was correctly inserted
- Shows sample of cleaned data
- Provides fallback to CSV if warehouse storage fails

The code follows these best practices:
- Efficient batch processing
- Proper error handling
- Data type preservation
- NULL value handling
- String sanitization
- Schema preservation


In [ ]:
# installation

%pip install pyspark delta-spark

     ━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/317.2 MB 203.9 MB/s eta 0:00:02

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 317.2/317.2 MB 154.8 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 317.2/317.2 MB 154.8 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 MB 5.0 MB/s eta 0:00:00


  Preparing metadata (setup.py) ... - \ done

...

 \ | /

 - \ | / - \ done
  Created wheel for pyspark: filename=pyspark-3.5.5-py2.py3-none-any.whl size=317747921 sha256=79151af543bfd8b32c63666ce05991363cf6bf6ce35a22514104a7c19a7feca7
  Stored in directory: /home/trusted-service-user/.cache/pip/wheels/0c/7f/b4/0e68c6d8d89d2e582e5498ad88616c16d7c19028680e9d3840
Successfully built pyspark


  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1.5.4.20240407
    Uninstalling pyspark-3.5.1.5.4.20240407:


      Successfully uninstalled pyspark-3.5.1.5.4.20240407


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import pyarrow as pa


def clean_spice_data(df):
    """
    Clean the Spice data with thorough null handling and data validation
    """
    print("Starting data cleaning process...")
    print(f"Original data shape: {df.shape}")
    print("\nOriginal columns:", df.columns.tolist())
    
    # Print detailed initial data quality check
    print("\nInitial data quality check:")
    for col in df.columns:
        total_rows = len(df)
        null_count = df[col].isna().sum()
        blank_count = df[col].astype(str).str.strip().isin(['', 'nan', 'None', 'NULL', ' ']).sum()
        unique_values = df[col].nunique()
        print(f"\nColumn '{col}':")
        print(f"- Data type: {df[col].dtype}")
        print(f"- Null count: {null_count} ({(null_count/total_rows)*100:.2f}%)")
        print(f"- Blank/Empty count: {blank_count} ({(blank_count/total_rows)*100:.2f}%)")
        print(f"- Unique values: {unique_values}")
        print(f"- Sample values: {df[col].dropna().head().tolist()}")
    
    # 1. Standardize column names
    print("\nStandardizing column names...")
    new_columns = {col: str(col).strip().lower().replace(' ', '_').replace('-', '_') 
                  for col in df.columns}
    df = df.rename(columns=new_columns)
    print("New column names:", df.columns.tolist())
    
    # 2. Remove unnecessary columns with validation
    columns_to_remove = [
        'dept', 'section', 'family', 'sub_family', 'brand_no', 
        'brand_principle', 'supplierno', 'mkt_code', 'mkt_name', 
        'month_2', '1', '2', '3', '4', '5'
    ]
    
    print("\nValidating columns to remove...")
    for col in columns_to_remove:
        if col in df.columns:
            print(f"- '{col}' found and will be removed")
            # Print sample of data being removed for validation
            print(f"  Sample data being removed: {df[col].head().tolist()}")
    
    columns_to_remove = [col for col in columns_to_remove if col in df.columns]
    if columns_to_remove:
        df = df.drop(columns=columns_to_remove)
        print(f"\nRemoved {len(columns_to_remove)} unnecessary columns")
        print("Remaining columns:", df.columns.tolist())
    
    # 3. Handle missing values with validation
    print("\nCleaning missing values...")
    
    # For numeric columns
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    for col in numeric_cols:
        before_count = df[col].isna().sum()
        mask = (df[col].isna() | 
                df[col].astype(str).str.strip().isin(['', 'nan', 'None', 'NULL', ' ']) |
                df[col].astype(str).str.contains('^\\s*$'))
        
        null_count = mask.sum()
        if null_count > 0:
            print(f"\nCleaning numeric column '{col}':")
            print(f"- Before cleaning: {before_count} nulls")
            df[col] = df[col].mask(mask, 0)
            after_count = df[col].isna().sum()
            print(f"- After cleaning: {after_count} nulls")
            print(f"- Replaced {null_count} values with 0")
            print(f"- Sample cleaned values: {df[col].head().tolist()}")
    
    # For text columns
    text_cols = df.select_dtypes(include=['object']).columns
    for col in text_cols:
        before_count = df[col].isna().sum()
        mask = (df[col].isna() | 
                df[col].astype(str).str.strip().isin(['', 'nan', 'None', 'NULL', ' ']) |
                df[col].astype(str).str.contains('^\\s*$'))
        
        null_count = mask.sum()
        if null_count > 0:
            print(f"\nCleaning text column '{col}':")
            print(f"- Before cleaning: {before_count} nulls/blanks")
            df[col] = df[col].mask(mask, "NULL")
            # Additional cleaning: strip whitespace
            df[col] = df[col].str.strip()
            after_count = df[col].isna().sum()
            print(f"- After cleaning: {after_count} nulls/blanks")
            print(f"- Replaced {null_count} values with 'NULL'")
            print(f"- Sample cleaned values: {df[col].head().tolist()}")
    
    # Final verification
    print("\nFinal data quality check:")
    all_clean = True
    for col in df.columns:
        null_count = df[col].isna().sum()
        blank_count = df[col].astype(str).str.strip().isin(['', 'nan', 'None', 'NULL', ' ']).sum()
        if null_count > 0 or blank_count > 0:
            all_clean = False
            print(f"WARNING - Column '{col}' still has issues:")
            print(f"- Null count: {null_count}")
            print(f"- Blank/Empty count: {blank_count}")
            print(f"- Sample values: {df[col].head().tolist()}")
    
    if all_clean:
        print("All columns have been successfully cleaned!")
    
    # Remove duplicates with validation
    rows_before = len(df)
    df = df.drop_duplicates()
    rows_after = len(df)
    if rows_before > rows_after:
        print(f"\nRemoved {rows_before - rows_after} duplicate rows")
        print(f"- Rows before: {rows_before}")
        print(f"- Rows after: {rows_after}")
    
    print(f"\nCleaning complete. Final data shape: {df.shape}")
    return df
   

# Connect to the warehouse
print("Connecting to the warehouse...")
conn = notebookutils.data.connect_to_artifact("DWSpice_Test", "8d3e063b-5552-4052-b919-22ef780dcf63", "Warehouse")

# Fetch the data and column information
print("Fetching data from Spice_Data_csv table...")
sql = """
SELECT * FROM [DWSpice_Test].[dbo].[Spice_Data_csv]
"""
cursor = conn.execute(sql)

# Store column information including types for later use
column_info = []
for col in cursor.description:
    column_name = col[0]
    column_type_code = col[1]  # This is the type code
    column_info.append((column_name, column_type_code))

# Fetch the actual data
columns = [column[0] for column in cursor.description]
data = cursor.fetchall()
columnar_data = list(zip(*data))


if len(columnar_data) > 0:
    # Convert to pandas DataFrame
    arrow_table = pa.Table.from_arrays([pa.array(col) for col in columnar_data], columns)
    df = arrow_table.to_pandas()
    print(f"Fetched {len(df)} rows and {len(df.columns)} columns")
    
    # Clean the data
    cleaned_df = clean_spice_data(df)
    
    # Display sample of cleaned data
    print("\nSample of cleaned data:")
    display(cleaned_df.head())
    
    # Store in memory for immediate use
    global cleaned_spice_data
    cleaned_spice_data = cleaned_df
    
    try:
        print("\nStoring cleaned data to lakehouse...")
        
        # Use ABFS path for the lakehouse
        abfs_path = "abfss://8d3e063b-5552-4052-b919-22ef780dcf63@onelake.dfs.fabric.microsoft.com/2d082e8f-8383-4582-83a0-a6858b56c5e4/Files"
        
        # Save as parquet file
        parquet_filename = "cleaned_spice_data.parquet"
        full_path = f"{abfs_path}/{parquet_filename}"
        
        # Save using pandas with ABFS path
        cleaned_df.to_parquet(full_path, index=False)
        print(f"Saved parquet file to: {full_path}")
        
        print("\nData successfully stored in lakehouse!")
        print("You can now access the cleaned data via:")
        print(f"1. The 'cleaned_spice_data' variable in memory")
        print(f"2. Parquet file at: {full_path}")
        
    except Exception as e:
        print(f"Error storing data to lakehouse: {str(e)}")
        # As a fallback, save to local CSV
        cleaned_df.to_csv("cleaned_spice_data.csv", index=False)
        print("Saved data to local CSV file as fallback")
        
    print("\nStep 2 (Data Cleaning) completed successfully!")
    
else:
    print('Empty table - no data to clean')

Connecting to the warehouse...
Fetching data from Spice_Data_csv table...
Fetched 88486 rows and 86 columns
Starting data cleaning process...
Original data shape: (88486, 86)

Original columns: ['Year', 'Month', 'Dept', 'Dept Name', 'Section', 'Section Name', 'Family', 'Family name', 'Sub Family', 'Sub Family Name', 'Brand No', 'Brand Principle', 'Brand Name', 'SupplierNo', 'SupplierName', 'Item Code', 'Item Bar Code', 'Item Name', 'MKT Code', 'MKT Name', '3110 - KRIV', '3111 - KHUB', '3120 - KTRM', '3123 - KMGA', '3126 - KGLR', '8122 - KJCN', '8124 - KSRT', '3118 - KSOF', '8128 - KXVLM', '8131 - KXNCM', '3133 - KKI1', '8115 - KXWGT', '8116 - KXNGN', '8139 - KXDIN', '3104 - KKEE', '8114 - KXGCM', '8111 - KXKI2', '8119 - KXKLM', '8117 - KXVYA', '8123 - KXCTH', '3112 - KBBY', '8151 - XKEG', '8140 - XKEF', '8121 - KXKEC', 'WEB Store Total QTY', 'WEB Store Total Sales', 'Total Qty', 'Total Sales', 'PRICE', 'TWO RIVERS', 'HUB', 'TRM', 'MEGA', 'GALLERIA', 'JUNCTION', 'SARIT', 'SOUTHFIELD', '


Storing cleaned data to lakehouse...
Saved parquet file to: abfss://8d3e063b-5552-4052-b919-22ef780dcf63@onelake.dfs.fabric.microsoft.com/2d082e8f-8383-4582-83a0-a6858b56c5e4/Files/cleaned_spice_data.parquet

Data successfully stored in lakehouse!
You can now access the cleaned data via:
1. The 'cleaned_spice_data' variable in memory
2. Parquet file at: abfss://8d3e063b-5552-4052-b919-22ef780dcf63@onelake.dfs.fabric.microsoft.com/2d082e8f-8383-4582-83a0-a6858b56c5e4/Files/cleaned_spice_data.parquet

Step 2 (Data Cleaning) completed successfully!


## unpivoting Data 
 Transform your data from a wide format (with store columns) to a long format (with store and amount columns) using unpivoting.

In [1]:
import pandas as pd
import numpy as np
import notebookutils.data
import pyarrow as pa

# This script transforms the Spice data by unpivoting store columns into a long format
# with 'store' and 'amount' columns for easier analysis
# The output is saved as a CSV file in the lakehouse for better compatibility with Microsoft Fabric

def get_store_name_mapping():
    """
    Return a mapping of store codes to store names
    """
    # Create a reverse mapping from codes to names
    code_to_name = {
        '3111 - khub': 'the_hub',
        '3110 - kriv': 'two_rivers',
        '3120 - ktrm': 'thika_road_mall',
        '8124 - ksrt': 'sarit_centre',
        '3123 - kmga': 'mega_mall',
        '3126 - kglr': 'galleria_mall',
        '8122 - kjcn': 'junction_mall',
        '8128 - kxvlm': 'village_market',
        '8139 - kxdin': 'diani',
        '8131 - kxncm': 'nyali',
        '3133 - kki1': 'kisumu_united',
        '3118 - ksof': 'south_field',
        '8111 - kxki2': 'kisumu_mega',
        '8115 - kxwgt': 'westgate',
        '8117 - kxvya': 'valley_arcade',
        '3112 - kbby': 'business_bay',
        '8116 - kxngn': 'nextgen',
        '8119 - kxklm': 'kilimani',
        '8114 - kxgcm': 'garden_city',
        '8140 - xkef': 'promenade',
        '3119 - kiru': 'ruiru',
        '8121 - kxkec': 'st_ellis',
        '8123 - kxcth': 'comet_house',
        '8151 - xkeg': 'gtc',
        '8148 - xkei': 'runda',
        '8142 - xkeh': 'rubis_makutano'
    }
    
    # Also create mappings for just the code part and just the short name part
    # This handles cases where the store column might be in different formats
    code_only_mapping = {}
    short_name_mapping = {}
    
    for code, name in code_to_name.items():
        parts = code.split(' - ')
        if len(parts) == 2:
            code_only = parts[0].strip()
            short_name = parts[1].strip()
            code_only_mapping[code_only] = name
            short_name_mapping[short_name] = name
    
    return code_to_name, code_only_mapping, short_name_mapping

def transform_spice_data(df):
    """
    Transform the Spice data by unpivoting store columns into a long format
    with 'store' and 'amount' columns for easier analysis in Microsoft Fabric
    """
    print("Starting Spice data transformation...")
    print(f"Original data shape: {df.shape}")
    
    # Print a few column names to understand the structure
    print("\nFirst few column names:")
    for i, col in enumerate(df.columns[:5]):
        print(f"{i}: {col}")
    
    # Get store name mappings
    code_to_name, code_only_mapping, short_name_mapping = get_store_name_mapping()
    
    # Identify which columns are common (non-store) columns and which are store columns
    # Common columns include metadata about products, totals, etc.
    # Store columns contain sales data for specific store locations
    
    # These are columns we know are NOT store columns (common columns)
    # Using standardized column names (lowercase with underscores)
    common_columns = [
        'year', 'month', 'family_name', 'sub_family_name',
        'brand_name', 'suppliername', 'item_code', 'item_bar_code', 'item_name',
        'web_store_total_qty', 'web_store_total_sales', 'price',
        'grammage_2', 'variants_2', 'type', 'category', 'super_category', 'packaging', 'brand'
    ]
    
    # Columns to explicitly drop
    columns_to_drop = ['dept_name', 'section_name', 'total_qty', 'total_sales']
    
    # Print the actual columns in the dataframe to verify
    print("\nActual columns in the dataframe:")
    print(df.columns.tolist())
    
    # First drop the columns we explicitly want to remove
    columns_to_drop_verified = []
    for col in columns_to_drop:
        if col in df.columns:
            columns_to_drop_verified.append(col)
        else:
            # Try case-insensitive match
            matches = [actual_col for actual_col in df.columns if actual_col.lower() == col.lower()]
            if matches:
                print(f"Found column to drop '{matches[0]}' for expected column '{col}'")
                columns_to_drop_verified.append(matches[0])
            else:
                print(f"Column to drop '{col}' not found in the dataframe")
    
    if columns_to_drop_verified:
        print(f"\nDropping {len(columns_to_drop_verified)} columns: {columns_to_drop_verified}")
        df = df.drop(columns=columns_to_drop_verified)
        print(f"Dataframe shape after dropping columns: {df.shape}")
    
    # Ensure we're using the correct column names by checking if they exist in the dataframe
    verified_common_columns = []
    missing_columns = []
    
    for col in common_columns:
        # Try exact match first
        if col in df.columns:
            verified_common_columns.append(col)
        else:
            # Try case-insensitive match
            matches = [actual_col for actual_col in df.columns if actual_col.lower() == col.lower()]
            if matches:
                print(f"Found column '{matches[0]}' for expected column '{col}'")
                verified_common_columns.append(matches[0])
            else:
                missing_columns.append(col)
    
    if missing_columns:
        print("\nWARNING: The following expected columns were not found in the dataframe:")
        for col in missing_columns:
            print(f"- {col}")
    
    # Update common_columns to use only verified columns
    common_columns = verified_common_columns
    print(f"\nVerified {len(common_columns)} common columns that exist in the dataframe")
    
    # Categorize columns into store columns and non-store columns (excluding total columns)
    non_store_columns = []
    store_columns = []
    
    # Define total columns with their corresponding value types
    # We'll verify these exist in the dataframe
    # Note: We're only keeping web_store_total columns as requested
    expected_total_columns = {
        'web_store_total_qty': 'qty',
        'web_store_total_sales': 'sales'
    }
    
    # Verify total columns exist in the dataframe
    total_columns = {}
    for col, value_type in expected_total_columns.items():
        # Try exact match first
        if col in df.columns:
            total_columns[col] = value_type
        else:
            # Try case-insensitive match
            matches = [actual_col for actual_col in df.columns if actual_col.lower() == col.lower()]
            if matches:
                print(f"Found total column '{matches[0]}' for expected column '{col}'")
                total_columns[matches[0]] = value_type
            else:
                print(f"WARNING: Expected total column '{col}' not found in the dataframe")
    
    print("\nIdentifying column types...")
    
    # Get the store name mapping to help identify store columns
    code_to_name, code_only_mapping, short_name_mapping = get_store_name_mapping()
    
    # Collect all possible store identifiers (codes and names)
    store_identifiers = set()
    for code in code_to_name.keys():
        store_identifiers.add(code.lower())
    for code in code_only_mapping.keys():
        store_identifiers.add(code.lower())
    for short_name in short_name_mapping.keys():
        store_identifiers.add(short_name.lower())
    for name in code_to_name.values():
        store_identifiers.add(name.lower())
    
    # Function to check if a column is a store column
    def is_store_column(col_name):
        col_lower = col_name.lower()
        # Check if the column contains any store identifier
        for identifier in store_identifiers:
            if identifier in col_lower:
                return True
        # Also check for numeric codes that might be store codes
        if any(code in col_lower for code in ['3110', '3111', '3120', '8122', '8124']):
            return True
        return False
    
    # Categorize columns
    for col in df.columns:
        if col in total_columns:
            print(f"Total column: {col}")
            # Total columns will be handled separately, not included in non_store_columns
        elif col in common_columns or any(common in col.lower() for common in ['total', 'price', 'web']):
            non_store_columns.append(col)
        elif is_store_column(col):
            store_columns.append(col)
        else:
            # If we're not sure, check if it looks like a price or quantity column
            # by examining the data type and values
            if df[col].dtype in [np.float64, np.int64] and not col.lower() in ['year', 'month', 'item_code', 'item_bar_code']:
                # Sample some values to see if they look like prices/quantities
                sample_vals = df[col].dropna().head(10).tolist()
                if sample_vals and all(val >= 0 for val in sample_vals):
                    print(f"Detected potential store column based on values: {col}")
                    store_columns.append(col)
                else:
                    non_store_columns.append(col)
            else:
                non_store_columns.append(col)
    
    print(f"\nIdentified {len(store_columns)} store columns")
    if store_columns:
        print(f"Sample store columns: {store_columns[:5]}")
    
    print(f"\nIdentified {len(non_store_columns)} non-store columns")
    print(f"Sample non-store columns: {non_store_columns[:5]}")
    
    # Perform the unpivot operation
    print("\nPerforming unpivot operation...")
    
    try:
        # First, create a copy of the dataframe with only non-store columns and store columns
        df_for_unpivot = df[non_store_columns + store_columns].copy()
        
        # Make sure 'price' is included in the non-store columns if it exists
        if 'price' in df.columns and 'price' not in non_store_columns:
            print("Adding 'price' column to non-store columns")
            non_store_columns.append('price')
            df_for_unpivot['price'] = df['price']
        
        # Use pandas melt function to unpivot store columns
        unpivoted_df = pd.melt(
            df_for_unpivot,
            id_vars=non_store_columns,
            value_vars=store_columns,
            var_name='store',
            value_name='amount'
        )
        
        # Drop rows with NaN or zero amounts
        print(f"Original unpivoted shape: {unpivoted_df.shape}")
        unpivoted_df = unpivoted_df.dropna(subset=['amount'])
        unpivoted_df = unpivoted_df[unpivoted_df['amount'] != 0]
        print(f"Shape after dropping NaN/zero amounts: {unpivoted_df.shape}")
        
        # Now create additional rows for the total columns
        print("\nAdding TOTAL store entries from total columns...")
        total_rows = []
        
        # For each row in the original dataframe
        for _, row in df.iterrows():
            # Create a base row with all the non-store columns
            base_row = {col: row[col] for col in non_store_columns}
            
            # Add 'web_store_total' entry if web store total columns exist
            if 'web_store_total_qty' in total_columns and 'web_store_total_sales' in total_columns:
                if not pd.isna(row['web_store_total_qty']) and row['web_store_total_qty'] != 0:
                    web_total_row = base_row.copy()
                    web_total_row['store'] = 'web_store_total'
                    web_total_row['amount'] = row['web_store_total_sales']  # Use sales as amount
                    web_total_row['qty'] = row['web_store_total_qty']  # Add qty as a separate column
                    
                    # Calculate unit price if both sales and qty are available
                    if not pd.isna(row['web_store_total_sales']) and not pd.isna(row['web_store_total_qty']) and row['web_store_total_qty'] > 0:
                        web_total_row['unit_price'] = row['web_store_total_sales'] / row['web_store_total_qty']
                    elif 'price' in row and not pd.isna(row['price']):
                        web_total_row['unit_price'] = row['price']
                    else:
                        web_total_row['unit_price'] = np.nan
                        
                    total_rows.append(web_total_row)
        
        # Create a dataframe from the total rows and append to the unpivoted dataframe
        if total_rows:
            total_df = pd.DataFrame(total_rows)
            print(f"Created {len(total_df)} total rows")
            
            # Add 'qty' column to unpivoted_df if it doesn't exist
            if 'qty' not in unpivoted_df.columns:
                unpivoted_df['qty'] = np.nan
                
            # Combine the unpivoted dataframe with the total dataframe
            unpivoted_df = pd.concat([unpivoted_df, total_df], ignore_index=True)
            print(f"Combined unpivoted data with total rows: {unpivoted_df.shape}")
        
        # Convert 'NULL' strings to actual None values
        unpivoted_df['amount'] = unpivoted_df['amount'].replace('NULL', None)
        
        # Convert amount to numeric, coercing errors to NaN
        unpivoted_df['amount'] = pd.to_numeric(unpivoted_df['amount'], errors='coerce')
        
        # Filter out rows with null or zero amounts
        unpivoted_df = unpivoted_df[unpivoted_df['amount'].notna() & (unpivoted_df['amount'] != 0)]
        
        # Add unit_price column if it doesn't exist
        if 'unit_price' not in unpivoted_df.columns:
            # First check if we have qty column to calculate unit price
            if 'qty' in unpivoted_df.columns:
                print("Calculating unit price from amount and qty")
                # Calculate unit price as amount/qty where qty > 0
                mask = (unpivoted_df['qty'].notna()) & (unpivoted_df['qty'] > 0)
                unpivoted_df.loc[mask, 'unit_price'] = unpivoted_df.loc[mask, 'amount'] / unpivoted_df.loc[mask, 'qty']
            
            # For rows without qty or with qty=0, use the price column if available
            if 'price' in unpivoted_df.columns:
                print("Using price column as fallback for unit price")
                mask = unpivoted_df['unit_price'].isna() | (unpivoted_df['unit_price'] == 0)
                unpivoted_df.loc[mask, 'unit_price'] = unpivoted_df.loc[mask, 'price']
            else:
                print("No price column available for unit price calculation")
        
        # Map store codes to store names
        print("\nMapping store codes to store names...")
        original_store_values = unpivoted_df['store'].unique()
        print(f"Original unique store values: {len(original_store_values)}")
        print(f"Sample original store values: {original_store_values[:5]}")
        
        # Function to map a store code to a store name
        def map_store_code_to_name(store_code):
            # Convert to lowercase for case-insensitive matching
            store_code_lower = store_code.lower() if isinstance(store_code, str) else str(store_code).lower()
            
            # Try exact match first
            if store_code_lower in code_to_name:
                return code_to_name[store_code_lower]
            
            # Try matching just the code part (e.g., '3111')
            for code in code_only_mapping:
                if code.lower() in store_code_lower:
                    return code_only_mapping[code]
            
            # Try matching just the short name part (e.g., 'khub')
            for short_name in short_name_mapping:
                if short_name.lower() in store_code_lower:
                    return short_name_mapping[short_name]
                
            # Special handling for common variations
            # Check if it's a store name with underscores instead of spaces
            for name in code_to_name.values():
                if name.lower() == store_code_lower or name.lower().replace('_', ' ') == store_code_lower:
                    return name
            
            # If no match found, return the original code but convert to lowercase with underscores
            # to maintain consistency with our naming convention
            return store_code_lower.replace(' ', '_')
        
        # Apply the mapping function to the store column
        unpivoted_df['store'] = unpivoted_df['store'].apply(map_store_code_to_name)
        
        # Check the results of the mapping
        mapped_store_values = unpivoted_df['store'].unique()
        print(f"Mapped unique store values: {len(mapped_store_values)}")
        print(f"Sample mapped store values: {mapped_store_values[:5]}")
        
        # Identify any store values that weren't properly mapped
        # These would be values that don't match any of our known store names
        known_store_names = set(code_to_name.values())
        unmapped_values = [val for val in mapped_store_values if val not in known_store_names and val != 'web_store_total']
        
        if unmapped_values:
            print("\nWARNING: Found unmapped store values:")
            for val in unmapped_values:
                print(f"- {val}")
            
            # Create a mapping for the unmapped values
            unmapped_mapping = {
                '3104___kkee': 'kee',
                'hub': 'the_hub',
                'trm': 'thika_road_mall',
                'mega': 'mega_mall',
                'galleria': 'galleria_mall',
                'junction': 'junction_mall',
                'sarit': 'sarit_centre',
                'southfield': 'south_field',
                'village': 'village_market',
                'west_gate': 'westgate',
                'next_gen': 'nextgen',
                'kee': 'kee',
                'commet_house': 'comet_house',
                'st._ellies': 'st_ellis'
            }
            
            # Apply the mapping to fix unmapped values
            print("\nApplying fixes for unmapped store values...")
            for unmapped, mapped in unmapped_mapping.items():
                # Check if this unmapped value exists in our data
                if unmapped in unmapped_values:
                    print(f"Mapping '{unmapped}' to '{mapped}'")
                    # Replace the unmapped value with the mapped value
                    unpivoted_df.loc[unpivoted_df['store'] == unmapped, 'store'] = mapped
        
        # Calculate the final shape of the unpivoted data
        print(f"Unpivoted data shape: {unpivoted_df.shape}")
        
        # Reorder columns to put store and amount in a logical position
        # Move 'store' and 'amount' to after 'item_name' if it exists
        cols = unpivoted_df.columns.tolist()
        
        # Find the position of item_name
        item_name_pos = -1
        for i, col in enumerate(cols):
            if col == 'item_name':
                item_name_pos = i
                break
        
        if item_name_pos >= 0:
            # Reorder columns
            new_cols = cols[:item_name_pos+1] + ['store', 'amount'] + \
                      [c for c in cols if c not in ['store', 'amount'] and c not in cols[:item_name_pos+1]]
            unpivoted_df = unpivoted_df[new_cols]
        
        print(f"Unpivoted data shape: {unpivoted_df.shape}")
        print("Unpivoting complete!")
        
        # Sample of unpivoted data with focus on store and amount columns
        print("\nSample of unpivoted data:")
        print("Columns in unpivoted data: {}".format(', '.join(unpivoted_df.columns)))
        
        # Get a sample of different stores to show variety
        print("\nSample rows from different stores:")
        # Get unique stores and sample up to 5 different ones
        unique_stores = unpivoted_df['store'].unique()[:5]
        sample_rows = []
        for store in unique_stores:
            # Get the first row for each store
            store_row = unpivoted_df[unpivoted_df['store'] == store].iloc[0]
            sample_rows.append(store_row)
        
        # Create a sample dataframe with different stores
        sample_df = pd.DataFrame(sample_rows)
        
        # Ensure store, amount, qty, and unit_price columns are displayed prominently
        cols_to_show = ['store', 'amount', 'qty', 'unit_price'] + [col for col in sample_df.columns if col not in ['store', 'amount', 'qty', 'unit_price']][:6]
        print(sample_df[cols_to_show].to_string(index=False))
        
        return unpivoted_df
        
    except Exception as e:
        print(f"Error during unpivot operation: {str(e)}")
        print("Could not unpivot the data. Returning original dataframe.")
        return df

# Function to load data from the lakehouse parquet file
def load_data():
    """
    Load the data directly from the warehouse using the same approach as workspace.py
    """
    try:
        # Connect to the warehouse
        print("Connecting to the warehouse...")
        conn = notebookutils.data.connect_to_artifact("DWSpice_Test", "8d3e063b-5552-4052-b919-22ef780dcf63", "Warehouse")
        
        # Fetch the data and column information
        print("Fetching data from Spice_Data_csv table...")
        sql = """
        SELECT * FROM [DWSpice_Test].[dbo].[Spice_Data_csv]
        """
        cursor = conn.execute(sql)
        
        # Store column information including types for later use
        column_info = []
        for col in cursor.description:
            column_name = col[0]
            column_type_code = col[1]  # This is the type code
            column_info.append((column_name, column_type_code))
        
        # Fetch the actual data
        columns = [column[0] for column in cursor.description]
        data = cursor.fetchall()
        columnar_data = list(zip(*data))
        
        if len(columnar_data) > 0:
            # Convert to pandas DataFrame using PyArrow for better performance
            import pyarrow as pa
            arrow_table = pa.Table.from_arrays([pa.array(col) for col in columnar_data], columns)
            df = arrow_table.to_pandas()
            print(f"Fetched {len(df)} rows and {len(df.columns)} columns")
            return df
        else:
            print("No data returned from the warehouse query")
            return None
    except Exception as e:
        print(f"Error loading data from warehouse: {str(e)}")
        # Fallback to parquet file if warehouse connection fails
        try:
            lakehouse_path = "abfss://8d3e063b-5552-4052-b919-22ef780dcf63@onelake.dfs.fabric.microsoft.com/2d082e8f-8383-4582-83a0-a6858b56c5e4/Files/cleaned_spice_data.parquet"
            print(f"Falling back to lakehouse parquet: {lakehouse_path}")
            df = pd.read_parquet(lakehouse_path)
            print(f"Successfully loaded data from lakehouse: {df.shape[0]} rows, {df.shape[1]} columns")
            return df
        except Exception as e2:
            print(f"Error loading data from lakehouse: {str(e2)}")
            return None

# Main execution
if __name__ == "__main__":
    # Load data from lakehouse
    df = load_data()
    
    # Perform the transformation
    transformed_df = transform_spice_data(df)
    
    # Print information about the transformed data
    print("\nTransformed data information:")
    print(f"- Total rows: {len(transformed_df)}")
    print(f"- Total columns: {len(transformed_df.columns)}")
    print("- Column names: {}".format(', '.join(transformed_df.columns[:10]) + '...' if len(transformed_df.columns) > 10 else ', '.join(transformed_df.columns)))
    
    # Show a sample of the data with store, amount, and unit price columns
    print("\nSample of transformed data (first 5 rows):")
    cols_to_show = ['store', 'amount', 'qty', 'unit_price', 'year', 'month', 'family_name', 'item_name', 'item_code']
    print(transformed_df[cols_to_show].head())
    
    # Save the transformed data
    # First save locally
    import os
    
    # Save as parquet locally first
    local_parquet_path = "transformed_spice_data.parquet"
    
    # Check if the file exists and remove it
    if os.path.exists(local_parquet_path):
        print(f"Removing existing local file: {local_parquet_path}")
        os.remove(local_parquet_path)
    
    # Save the parquet locally
    print(f"Saving transformed data to local parquet: {local_parquet_path}")
    transformed_df.to_parquet(local_parquet_path, index=False)
    print("Local parquet saved successfully!")
    
    # Also print some information about the saved data
    print(f"\nSaved data summary:")
    print(f"- Total rows: {len(transformed_df)}")
    print(f"- Total columns: {len(transformed_df.columns)}")
    print(f"- Store values: {sorted(transformed_df['store'].unique())}")
    print("- Sample data:")
    print(transformed_df[['store', 'amount', 'year', 'month', 'item_name']].head().to_string())
    
    # Now save to lakehouse
    lakehouse_path = "abfss://8d3e063b-5552-4052-b919-22ef780dcf63@onelake.dfs.fabric.microsoft.com/2d082e8f-8383-4582-83a0-a6858b56c5e4/Files"
    
    # Save as parquet in the lakehouse
    parquet_path = f"{lakehouse_path}/transformed_spice_data.parquet"
    print(f"\nSaving transformed data to lakehouse parquet: {parquet_path}")
    
    # We can't directly check if the file exists in the lakehouse, but we can try to delete it first
    # using a try-except block to handle the case where it doesn't exist
    try:
        print(f"Attempting to remove existing file in lakehouse if it exists: {parquet_path}")
        # We'll use pandas to write an empty dataframe to overwrite the file
        pd.DataFrame().to_parquet(parquet_path, index=False)
        print("Existing file removed or overwritten")
    except Exception as e:
        print(f"Note: {str(e)}")
        print("Proceeding with saving the new file")
    
    # Now save the actual data
    transformed_df.to_parquet(parquet_path, index=False)
    print("Lakehouse parquet data saved successfully!")


Connecting to the warehouse...
Fetching data from Spice_Data_csv table...
Fetched 88486 rows and 86 columns
Starting Spice data transformation...
Original data shape: (88486, 86)

First few column names:
0: Year
1: Month
2: Dept
3: Dept Name
4: Section

Actual columns in the dataframe:
['Year', 'Month', 'Dept', 'Dept Name', 'Section', 'Section Name', 'Family', 'Family name', 'Sub Family', 'Sub Family Name', 'Brand No', 'Brand Principle', 'Brand Name', 'SupplierNo', 'SupplierName', 'Item Code', 'Item Bar Code', 'Item Name', 'MKT Code', 'MKT Name', '3110 - KRIV', '3111 - KHUB', '3120 - KTRM', '3123 - KMGA', '3126 - KGLR', '8122 - KJCN', '8124 - KSRT', '3118 - KSOF', '8128 - KXVLM', '8131 - KXNCM', '3133 - KKI1', '8115 - KXWGT', '8116 - KXNGN', '8139 - KXDIN', '3104 - KKEE', '8114 - KXGCM', '8111 - KXKI2', '8119 - KXKLM', '8117 - KXVYA', '8123 - KXCTH', '3112 - KBBY', '8151 - XKEG', '8140 - XKEF', '8121 - KXKEC', 'WEB Store Total QTY', 'WEB Store Total Sales', 'Total Qty', 'Total Sales', 

trying a different way to unpivot the data 

In [1]:
import pandas as pd
import numpy as np
import notebookutils.data
import pyarrow as pa

# This script transforms the Spice data by unpivoting store columns into a long format
# with 'store' and 'amount' columns for easier analysis
# The output is saved as a CSV file in the lakehouse for better compatibility with Microsoft Fabric

def standardize_column_names(df):
    """
    Standardize column names and handle common variations in column names
    """
    # First, standardize all column names to lowercase with underscores
    new_columns = {col: str(col).strip().lower().replace(' ', '_').replace('-', '_') for col in df.columns}
    df = df.rename(columns=new_columns)
    
    # Define common column name variations and their standard names
    column_variations = {
        'year_': 'year',
        'yr': 'year',
        'month_': 'month',
        'mo': 'month',
        'family': 'family_name',
        'subfamily': 'sub_family_name',
        'sub_family': 'sub_family_name',
        'brand': 'brand_name',
        'supplier': 'suppliername',
        'supplier_name': 'suppliername',
        'item_number': 'item_code',
        'item_no': 'item_code',
        'barcode': 'item_bar_code',
        'bar_code': 'item_bar_code',
        'description': 'item_name',
        'item_description': 'item_name',
        'unit_price': 'price',
        'selling_price': 'price'
    }
    
    # Check for variations and standardize them
    for col in df.columns:
        for variation, standard in column_variations.items():
            if col == variation or col.startswith(variation + '_') or col.endswith('_' + variation):
                print(f"Standardizing column name: '{col}' -> '{standard}'")
                df = df.rename(columns={col: standard})
                break
    
    return df

def get_store_name_mapping():
    """
    Return a mapping of store codes to store names
    """
    # Create a reverse mapping from codes to names
    code_to_name = {
        '3111 - khub': 'the_hub',
        '3110 - kriv': 'two_rivers',
        '3120 - ktrm': 'thika_road_mall',
        '8124 - ksrt': 'sarit_centre',
        '3123 - kmga': 'mega_mall',
        '3126 - kglr': 'galleria_mall',
        '8122 - kjcn': 'junction_mall',
        '8128 - kxvlm': 'village_market',
        '8139 - kxdin': 'diani',
        '8131 - kxncm': 'nyali',
        '3133 - kki1': 'kisumu_united',
        '3118 - ksof': 'south_field',
        '8111 - kxki2': 'kisumu_mega',
        '8115 - kxwgt': 'westgate',
        '8117 - kxvya': 'valley_arcade',
        '3112 - kbby': 'business_bay',
        '8116 - kxngn': 'nextgen',
        '8119 - kxklm': 'kilimani',
        '8114 - kxgcm': 'garden_city',
        '8140 - xkef': 'promenade',
        '3119 - kiru': 'ruiru',
        '8121 - kxkec': 'st_ellis',
        '8123 - kxcth': 'comet_house',
        '8151 - xkeg': 'gtc',
        '8148 - xkei': 'runda',
        '8142 - xkeh': 'rubis_makutano'
    }
    
    # Also create mappings for just the code part and just the short name part
    # This handles cases where the store column might be in different formats
    code_only_mapping = {}
    short_name_mapping = {}
    
    for code, name in code_to_name.items():
        parts = code.split(' - ')
        if len(parts) == 2:
            code_only = parts[0].strip()
            short_name = parts[1].strip()
            code_only_mapping[code_only] = name
            short_name_mapping[short_name] = name
    
    return code_to_name, code_only_mapping, short_name_mapping

def transform_spice_data(df):
    """
    Transform the Spice data by unpivoting store columns into a long format
    with 'store' and 'amount' columns for easier analysis in Microsoft Fabric
    """
    print("Starting Spice data transformation...")
    print(f"Original data shape: {df.shape}")
    
    # Get store name mappings
    code_to_name, code_only_mapping, short_name_mapping = get_store_name_mapping()
    
    # These are columns we know are NOT store columns (common columns)
    # Using standardized column names (lowercase with underscores)
    common_columns = [
        'year', 'month', 'family_name', 'sub_family_name',
        'brand_name', 'suppliername', 'item_code', 'item_bar_code', 'item_name',
        'web_store_total_qty', 'web_store_total_sales', 'price',
        'grammage_2', 'variants_2', 'type', 'category', 'super_category', 'packaging', 'brand'
    ]
    
    # Columns to explicitly drop
    columns_to_drop = ['dept_name', 'section_name', 'total_qty', 'total_sales']
    
    # Drop columns we explicitly want to remove (case-insensitive)
    columns_to_drop_verified = []
    for col in columns_to_drop:
        # Try exact match first, then case-insensitive match
        if col in df.columns:
            columns_to_drop_verified.append(col)
        else:
            matches = [actual_col for actual_col in df.columns if actual_col.lower() == col.lower()]
            if matches:
                columns_to_drop_verified.append(matches[0])
    
    if columns_to_drop_verified:
        print(f"\nDropping {len(columns_to_drop_verified)} columns: {columns_to_drop_verified}")
        df = df.drop(columns=columns_to_drop_verified)
        print(f"Dataframe shape after dropping columns: {df.shape}")
    
    # Ensure we're using the correct column names by checking if they exist in the dataframe
    verified_common_columns = []
    missing_columns = []
    
    for col in common_columns:
        # Try exact match first
        if col in df.columns:
            verified_common_columns.append(col)
        else:
            # Try case-insensitive match
            matches = [actual_col for actual_col in df.columns if actual_col.lower() == col.lower()]
            if matches:
                print(f"Found column '{matches[0]}' for expected column '{col}'")
                verified_common_columns.append(matches[0])
            else:
                missing_columns.append(col)
    
    if missing_columns:
        print("\nWARNING: The following expected columns were not found in the dataframe:")
        for col in missing_columns:
            print(f"- {col}")
    
    # Update common_columns to use only verified columns
    common_columns = verified_common_columns
    print(f"\nVerified {len(common_columns)} common columns that exist in the dataframe")
    
    # Categorize columns into store columns and non-store columns
    non_store_columns = []
    store_columns = []
    
    # Define web store columns we want to keep
    expected_total_columns = {
        'web_store_total_qty': 'qty',
        'web_store_total_sales': 'sales'
    }
    
    # Find web store columns (case-insensitive)
    total_columns = {}
    for col in df.columns:
        for web_col, value_type in expected_total_columns.items():
            if col.lower() == web_col.lower():
                print(f"Found web store column: {col}")
                total_columns[col] = value_type
                break
    
    print("\nIdentifying store columns...")
    
    # Get store identifiers from the mapping function
    code_to_name, code_only_mapping, short_name_mapping = get_store_name_mapping()
    
    # Create a set of all possible store identifiers (codes and names)
    store_identifiers = set()
    for mapping in [code_to_name.keys(), code_only_mapping.keys(), short_name_mapping.keys(), code_to_name.values()]:
        store_identifiers.update([s.lower() for s in mapping])
    
    # Add common store codes that might be in column names
    store_identifiers.update(['3110', '3111', '3120', '8122', '8124'])
    
    # Categorize columns
    for col in df.columns:
        # Skip columns we've already identified
        if col in total_columns or col in common_columns:
            non_store_columns.append(col)
            continue
            
        # Check if it's a store column by looking for store identifiers
        col_lower = col.lower()
        is_store = False
        
        # Check if column contains any store identifier
        for identifier in store_identifiers:
            if identifier in col_lower:
                store_columns.append(col)
                is_store = True
                break
                
        # If not identified as a store, check if it's numeric (potential store column)
        if not is_store:
            if pd.api.types.is_numeric_dtype(df[col]) and not col.lower() in ['year', 'month', 'item_code', 'item_bar_code']:
                # If it has positive values, it's likely a store column
                if df[col].dropna().gt(0).any():
                    store_columns.append(col)
                else:
                    non_store_columns.append(col)
            else:
                non_store_columns.append(col)
    
    print(f"\nIdentified {len(store_columns)} store columns")
    if store_columns:
        print(f"Sample store columns: {store_columns[:5]}")
    
    print(f"\nIdentified {len(non_store_columns)} non-store columns")
    print(f"Sample non-store columns: {non_store_columns[:5]}")
    
    # Perform the unpivot operation
    print("\nPerforming unpivot operation...")
    
    try:
        # First, create a copy of the dataframe with only non-store columns and store columns
        df_for_unpivot = df[non_store_columns + store_columns].copy()
        
        # Make sure 'price' is included in the non-store columns if it exists
        if 'price' in df.columns and 'price' not in non_store_columns:
            print("Adding 'price' column to non-store columns")
            non_store_columns.append('price')
            df_for_unpivot['price'] = df['price']
        
        # Use pandas melt function to unpivot store columns
        unpivoted_df = pd.melt(
            df_for_unpivot,
            id_vars=non_store_columns,
            value_vars=store_columns,
            var_name='store',
            value_name='amount'
        )
        
        # Drop rows with NaN or zero amounts
        print(f"Original unpivoted shape: {unpivoted_df.shape}")
        unpivoted_df = unpivoted_df.dropna(subset=['amount'])
        unpivoted_df = unpivoted_df[unpivoted_df['amount'] != 0]
        print(f"Shape after dropping NaN/zero amounts: {unpivoted_df.shape}")
        
        # We're not creating additional rows for the web store columns
        # as per user request, we'll leave them as they are
        print("\nNot adding web store entries as separate rows as requested.")
        
        # Convert 'NULL' strings to actual None values
        unpivoted_df['amount'] = unpivoted_df['amount'].replace('NULL', None)
        
        # Convert amount to numeric, coercing errors to NaN
        unpivoted_df['amount'] = pd.to_numeric(unpivoted_df['amount'], errors='coerce')
        
        # Filter out rows with null or zero amounts
        unpivoted_df = unpivoted_df[unpivoted_df['amount'].notna() & (unpivoted_df['amount'] != 0)]
        
        # Only keep amount and unit_price for unpivoted data, no qty
        print("\nProcessing unit_price field...")
        
        # Add unit_price column if it doesn't exist
        if 'unit_price' not in unpivoted_df.columns:
            print("Adding unit_price column")
            unpivoted_df['unit_price'] = np.nan
        
        # For unpivoted data, use price column as unit_price if available
        if 'price' in unpivoted_df.columns:
            price_mask = unpivoted_df['unit_price'].isna() & unpivoted_df['price'].notna()
            print(f"Using price column for {price_mask.sum()} rows without unit_price")
            unpivoted_df.loc[price_mask, 'unit_price'] = unpivoted_df.loc[price_mask, 'price']
        
        # Print statistics about unit_price
        print(f"Rows with unit_price values: {unpivoted_df['unit_price'].notna().sum()} ({unpivoted_df['unit_price'].notna().sum()/len(unpivoted_df)*100:.2f}%)")
        
        # Map store codes to store names
        print("\nMapping store codes to store names...")
        original_store_values = unpivoted_df['store'].unique()
        print(f"Original unique store values: {len(original_store_values)}")
        print(f"Sample original store values: {original_store_values[:5]}")
        
        # Function to map a store code to a store name
        def map_store_code_to_name(store_code):
            # Convert to lowercase for case-insensitive matching
            store_code_lower = store_code.lower() if isinstance(store_code, str) else str(store_code).lower()
            
            # Try exact match first
            if store_code_lower in code_to_name:
                return code_to_name[store_code_lower]
            
            # Try matching just the code part (e.g., '3111')
            for code in code_only_mapping:
                if code.lower() in store_code_lower:
                    return code_only_mapping[code]
            
            # Try matching just the short name part (e.g., 'khub')
            for short_name in short_name_mapping:
                if short_name.lower() in store_code_lower:
                    return short_name_mapping[short_name]
                
            # Special handling for common variations
            # Check if it's a store name with underscores instead of spaces
            for name in code_to_name.values():
                if name.lower() == store_code_lower or name.lower().replace('_', ' ') == store_code_lower:
                    return name
            
            # If no match found, return the original code but convert to lowercase with underscores
            # to maintain consistency with our naming convention
            return store_code_lower.replace(' ', '_')
        
        # Apply the mapping function to the store column
        unpivoted_df['store'] = unpivoted_df['store'].apply(map_store_code_to_name)
        
        # Check the results of the mapping
        mapped_store_values = unpivoted_df['store'].unique()
        print(f"Mapped unique store values: {len(mapped_store_values)}")
        print(f"Sample mapped store values: {mapped_store_values[:5]}")
        
        # Identify any store values that weren't properly mapped
        # These would be values that don't match any of our known store names
        known_store_names = set(code_to_name.values())
        unmapped_values = [val for val in mapped_store_values if val not in known_store_names and val != 'web_store']
        
        if unmapped_values:
            print("\nWARNING: Found unmapped store values:")
            for val in unmapped_values:
                print(f"- {val}")
            
            # Create a mapping for the unmapped values
            unmapped_mapping = {
                # Code-based variations
                '3104___kkee': 'kee',
                '3104_kkee': 'kee',
                '3104': 'kee',
                'kkee': 'kee',
                # Store name variations
                'hub': 'the_hub',
                'the_hub': 'the_hub',
                'trm': 'thika_road_mall',
                'thika_road': 'thika_road_mall',
                'mega': 'mega_mall',
                'galleria': 'galleria_mall',
                'junction': 'junction_mall',
                'sarit': 'sarit_centre',
                'sarit_center': 'sarit_centre',
                'southfield': 'south_field',
                'south_field': 'south_field',
                'village': 'village_market',
                'village_market': 'village_market',
                'west_gate': 'westgate',
                'westgate': 'westgate',
                'next_gen': 'nextgen',
                'nextgen': 'nextgen',
                'kee': 'kee',
                'commet_house': 'comet_house',
                'comet_house': 'comet_house',
                'st._ellies': 'st_ellis',
                'st_ellies': 'st_ellis',
                'st._ellis': 'st_ellis',
                'st_ellis': 'st_ellis',
                # Add any other variations you've observed
                'kisumu': 'kisumu_united',
                'kisumu_united': 'kisumu_united',
                'kisumu_mega': 'kisumu_mega',
                'diani': 'diani',
                'nyali': 'nyali',
                'garden_city': 'garden_city',
                'garden': 'garden_city',
                'business_bay': 'business_bay',
                'business': 'business_bay',
                'gtc': 'gtc',
                'promenade': 'promenade',
                'valley_arcade': 'valley_arcade',
                'valley': 'valley_arcade',
                'ruiru': 'ruiru',
                'two_rivers': 'two_rivers',
                'two_river': 'two_rivers',
                'rivers': 'two_rivers'
            }
            
            # Apply the mapping to fix unmapped values
            print("\nApplying fixes for unmapped store values...")
            for unmapped, mapped in unmapped_mapping.items():
                # Check if this unmapped value exists in our data
                if unmapped in unmapped_values:
                    print(f"Mapping '{unmapped}' to '{mapped}'")
                    # Replace the unmapped value with the mapped value
                    unpivoted_df.loc[unpivoted_df['store'] == unmapped, 'store'] = mapped
        
        # Calculate the final shape of the unpivoted data
        print(f"Unpivoted data shape: {unpivoted_df.shape}")
        
        # Reorder columns to put store and amount in a logical position
        # Move 'store' and 'amount' to after 'item_name' if it exists
        cols = unpivoted_df.columns.tolist()
        
        # Find the position of item_name
        item_name_pos = -1
        for i, col in enumerate(cols):
            if col == 'item_name':
                item_name_pos = i
                break
        
        if item_name_pos >= 0:
            # Reorder columns
            new_cols = cols[:item_name_pos+1] + ['store', 'amount'] + \
                      [c for c in cols if c not in ['store', 'amount'] and c not in cols[:item_name_pos+1]]
            unpivoted_df = unpivoted_df[new_cols]
        
        print(f"Unpivoted data shape: {unpivoted_df.shape}")
        print("Unpivoting complete!")
        
        # Sample of unpivoted data with focus on store and amount columns
        print("\nSample of unpivoted data:")
        print("Columns in unpivoted data: {}".format(', '.join(unpivoted_df.columns)))
        
        # Get a sample of different stores to show variety
        print("\nSample rows from different stores:")
        # Get unique stores and sample up to 5 different ones
        unique_stores = unpivoted_df['store'].unique()[:5]
        sample_rows = []
        for store in unique_stores:
            # Get the first row for each store
            store_row = unpivoted_df[unpivoted_df['store'] == store].iloc[0]
            sample_rows.append(store_row)
        
        # Create a sample dataframe with different stores
        sample_df = pd.DataFrame(sample_rows)
        
        # Ensure store, amount, and unit_price columns are displayed prominently if they exist
        base_cols = ['store', 'amount', 'unit_price']
        # Only include columns that actually exist in the dataframe
        available_cols = [col for col in base_cols if col in sample_df.columns]
        other_cols = [col for col in sample_df.columns if col not in base_cols][:6]
        cols_to_show = available_cols + other_cols
        print(sample_df[cols_to_show].to_string(index=False))
        
        return unpivoted_df
        
    except Exception as e:
        print(f"Error during unpivot operation: {str(e)}")
        print("Could not unpivot the data. Returning original dataframe.")
        return df

# Function to load data from the lakehouse parquet file
def load_data():
    """
    Load the data directly from the warehouse using the same approach as workspace.py
    """
    try:
        # Connect to the warehouse
        print("Connecting to the warehouse...")
        conn = notebookutils.data.connect_to_artifact("DWSpice_Test", "8d3e063b-5552-4052-b919-22ef780dcf63", "Warehouse")
        
        # Fetch the data and column information
        print("Fetching data from Spice_Data_csv table...")
        sql = """
        SELECT * FROM [DWSpice_Test].[dbo].[Spice_Data_csv]
        """
        cursor = conn.execute(sql)
        
        # Store column information including types for later use
        column_info = []
        for col in cursor.description:
            column_name = col[0]
            column_type_code = col[1]  # This is the type code
            column_info.append((column_name, column_type_code))
        
        # Fetch the actual data
        columns = [column[0] for column in cursor.description]
        data = cursor.fetchall()
        columnar_data = list(zip(*data))
        
        if len(columnar_data) > 0:
            # Convert to pandas DataFrame using PyArrow for better performance
            import pyarrow as pa
            arrow_table = pa.Table.from_arrays([pa.array(col) for col in columnar_data], columns)
            df = arrow_table.to_pandas()
            print(f"Fetched {len(df)} rows and {len(df.columns)} columns")
            
            # Standardize column names using our helper function
            print("\nStandardizing column names...")
            print("Original columns:", df.columns.tolist())
            df = standardize_column_names(df)
            print("Standardized columns:", df.columns.tolist())
            
            # Check for required columns and add them if missing
            required_columns = [
                'year', 'month', 'family_name', 'sub_family_name',
                'brand_name', 'suppliername', 'item_code', 'item_bar_code', 'item_name',
                'price'
            ]
            
            for col in required_columns:
                # Check if the column exists (case-insensitive)
                col_matches = [c for c in df.columns if c.lower() == col.lower()]
                if not col_matches:
                    print(f"WARNING: Required column '{col}' not found. Adding empty column.")
                    df[col] = np.nan
                elif col_matches[0] != col:
                    # If the column exists but with different casing, standardize it
                    print(f"Renaming column '{col_matches[0]}' to '{col}'")
                    df = df.rename(columns={col_matches[0]: col})
            
            # Print data quality information
            print("\nData quality check after standardization:")
            for col in required_columns:
                if col in df.columns:
                    null_count = df[col].isna().sum()
                    print(f"Column '{col}': {null_count} null values ({(null_count/len(df))*100:.2f}%)")
                else:
                    print(f"Column '{col}' still missing after standardization attempt")
            return df
        else:
            print("No data returned from the warehouse query")
            return None
    except Exception as e:
        print(f"Error loading data from warehouse: {str(e)}")
        # Fallback to parquet file if warehouse connection fails
        try:
            lakehouse_path = "abfss://8d3e063b-5552-4052-b919-22ef780dcf63@onelake.dfs.fabric.microsoft.com/2d082e8f-8383-4582-83a0-a6858b56c5e4/Files/cleaned_spice_data.parquet"
            print(f"Falling back to lakehouse parquet: {lakehouse_path}")
            df = pd.read_parquet(lakehouse_path)
            print(f"Successfully loaded data from lakehouse: {df.shape[0]} rows, {df.shape[1]} columns")
            return df
        except Exception as e2:
            print(f"Error loading data from lakehouse: {str(e2)}")
            return None

# Main execution
if __name__ == "__main__":
    # Load data from lakehouse
    df = load_data()
    
    # Perform the transformation
    transformed_df = transform_spice_data(df)
    
    # Remove qty column if it exists as per user request
    if 'qty' in transformed_df.columns:
        print("Removing 'qty' column as requested")
        transformed_df = transformed_df.drop(columns=['qty'])
    
    # Print information about the transformed data
    print("\nTransformed data information:")
    print(f"- Total rows: {len(transformed_df)}")
    print(f"- Total columns: {len(transformed_df.columns)}")
    print("- Column names: {}".format(', '.join(transformed_df.columns[:10]) + '...' if len(transformed_df.columns) > 10 else ', '.join(transformed_df.columns)))
    
    # Print statistics about key columns
    print("\nKey column statistics:")
    for col in ['amount', 'unit_price', 'price']:
        if col in transformed_df.columns:
            non_null = transformed_df[col].notna().sum()
            percent = (non_null / len(transformed_df)) * 100
            print(f"- {col}: {non_null} non-null values ({percent:.2f}%)")
            if non_null > 0:
                print(f"  Min: {transformed_df[col].min()}, Max: {transformed_df[col].max()}, Mean: {transformed_df[col].mean():.2f}")
    
    # Print store statistics if store column exists
    print("\nStore statistics:")
    if 'store' in transformed_df.columns:
        store_counts = transformed_df['store'].value_counts()
        print(f"- Total unique stores: {len(store_counts)}")
        print("- Top 5 stores by row count:")
        for store, count in store_counts.head(5).items():
            print(f"  {store}: {count} rows ({(count/len(transformed_df))*100:.2f}%)")
    else:
        print("- No 'store' column found in the transformed data")
    
    # Show a sample of the data with available columns
    print("\nSample of transformed data (first 5 rows):")
    # Determine which columns to show based on what's available
    available_cols = ['year', 'month', 'family_name', 'item_name', 'item_code']
    if 'store' in transformed_df.columns:
        available_cols = ['store'] + available_cols
    if 'amount' in transformed_df.columns:
        available_cols.insert(1 if 'store' in transformed_df.columns else 0, 'amount')
    if 'unit_price' in transformed_df.columns:
        available_cols.insert(2 if 'store' in transformed_df.columns else 1, 'unit_price')
    
    # Only include columns that actually exist in the dataframe
    cols_to_show = [col for col in available_cols if col in transformed_df.columns]
    print(transformed_df[cols_to_show].head())
    
    # Save the transformed data
    # First save locally
    import os
    
    # Save as parquet locally first
    local_parquet_path = "transformed_spice_data.parquet"
    
    # Check if the file exists and remove it
    if os.path.exists(local_parquet_path):
        print(f"Removing existing local file: {local_parquet_path}")
        os.remove(local_parquet_path)
    
    # Save the parquet locally
    print(f"Saving transformed data to local parquet: {local_parquet_path}")
    transformed_df.to_parquet(local_parquet_path, index=False)
    print("Local parquet saved successfully!")
    
    # Also print some information about the saved data
    print(f"\nSaved data summary:")
    print(f"- Total rows: {len(transformed_df)}")
    print(f"- Total columns: {len(transformed_df.columns)}")
    if 'store' in transformed_df.columns:
        print(f"- Store values: {sorted(transformed_df['store'].unique())}")
    else:
        print("- No 'store' column in transformed data")
    print("- Sample data:")
    # Select columns that exist in the dataframe
    display_cols = [col for col in ['store', 'amount', 'year', 'month', 'item_name'] if col in transformed_df.columns]
    print(transformed_df[display_cols].head().to_string())
    
    # Now save to lakehouse
    lakehouse_path = "abfss://8d3e063b-5552-4052-b919-22ef780dcf63@onelake.dfs.fabric.microsoft.com/f22d5105-68d1-4837-a8ad-7ed0b7028bc5/Files"
    
    # Save as parquet in the lakehouse
    parquet_path = f"{lakehouse_path}/transformed_spice_data.parquet"
    print(f"\nSaving transformed data to lakehouse parquet: {parquet_path}")
    
    # We can't directly check if the file exists in the lakehouse, but we can try to delete it first
    # using a try-except block to handle the case where it doesn't exist
    try:
        print(f"Attempting to remove existing file in lakehouse if it exists: {parquet_path}")
        # We'll use pandas to write an empty dataframe to overwrite the file
        pd.DataFrame().to_parquet(parquet_path, index=False)
        print("Existing file removed or overwritten")
    except Exception as e:
        print(f"Note: {str(e)}")
        print("Proceeding with saving the new file")
    
    # Now save the actual data
    transformed_df.to_parquet(parquet_path, index=False)
    print("Lakehouse parquet data saved successfully!")


Connecting to the warehouse...
Fetching data from Spice_Data_csv table...
Fetched 88486 rows and 86 columns

Standardizing column names...
Original columns: ['Year', 'Month', 'Dept', 'Dept Name', 'Section', 'Section Name', 'Family', 'Family name', 'Sub Family', 'Sub Family Name', 'Brand No', 'Brand Principle', 'Brand Name', 'SupplierNo', 'SupplierName', 'Item Code', 'Item Bar Code', 'Item Name', 'MKT Code', 'MKT Name', '3110 - KRIV', '3111 - KHUB', '3120 - KTRM', '3123 - KMGA', '3126 - KGLR', '8122 - KJCN', '8124 - KSRT', '3118 - KSOF', '8128 - KXVLM', '8131 - KXNCM', '3133 - KKI1', '8115 - KXWGT', '8116 - KXNGN', '8139 - KXDIN', '3104 - KKEE', '8114 - KXGCM', '8111 - KXKI2', '8119 - KXKLM', '8117 - KXVYA', '8123 - KXCTH', '3112 - KBBY', '8151 - XKEG', '8140 - XKEF', '8121 - KXKEC', 'WEB Store Total QTY', 'WEB Store Total Sales', 'Total Qty', 'Total Sales', 'PRICE', 'TWO RIVERS', 'HUB', 'TRM', 'MEGA', 'GALLERIA', 'JUNCTION', 'SARIT', 'SOUTHFIELD', 'VILLAGE', 'NYALI', 'KISUMU UNITED', '